In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#data from https://www.kaggle.com/ronitf/heart-disease-uci
df = pd.read_csv("C:/Users/Tim/Desktop/projdata/heartdisease/heart.csv")
df.describe()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


FileNotFoundError: ignored

In [ ]:
df['sex']=df['sex'].replace(
    [0,1],['female','male']
)
df['chestpain']=df['chestpain'].replace(
    [0,1,2,3],['typical','atypical','nonang','asymp']
)
df['fastingbloodsugar']=df['fastingbloodsugar'].replace(
    [0,1],['<120','>120']
)
df['restecg']=df['restecg'].replace(
    [0,1,2],['normal','waveabn','venthyper']
)
df['exerciseangina']=df['exerciseangina'].replace(
    [0,1],['no','yes']
)
df['stslope']=df['stslope'].replace(
    [0,1,2],['up','flat','down']
)
df['thalassemia']=df['thalassemia'].replace(
    [1,2,3],['normal','fixed','reversable']
)
df=pd.get_dummies(df,drop_first=True)
df.head()
y = df['target'].values
xorig = df.drop(['target'],axis=1)
#normalize
from sklearn import preprocessing
xorig = xorig.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(xorig)
x = pd.DataFrame(x_scaled)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state=123)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators= 1000, random_state=123)
rf.fit(X_train,y_train)

ypred = rf.predict(X_test)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(
    estimator = rf, 
    param_distributions = random_grid, 
    n_iter = 100, 
    cv = 5, 
    verbose=2, 
    random_state=123, 
    n_jobs = -1
)
rf_random.fit(X_train, y_train)

rf_random.best_params_

def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    #mape = 100 * np.mean(errors / y_test)
    #accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

baserf = RandomForestClassifier(n_estimators = 1000, random_state = 123)
baserf.fit(X_train, y_train)
base_accuracy = evaluate(baserf, X_test, y_test)
baserf.score(X_test,y_test)

bestrf = rf_random.best_estimator_
random_accuracy = evaluate(bestrf, X_test, y_test)
bestrf.score(X_test,y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [70, 80, 90, 100],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [1000, 1200, 1400, 2000]
}

rf = RandomForestClassifier()
gs = GridSearchCV(
    estimator = rf, 
    param_grid = param_grid, 
    cv = 5, 
    n_jobs = -1, 
    verbose = 2
)

gs.fit(X_train, y_train)
gs.best_params_

bestgrid = gs.best_estimator_
grid_accuracy = evaluate(bestgrid, X_test, y_test)
bestgrid.score(X_test,y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))


In [ ]:
import shap
#model with gs best params
model = RandomForestClassifier(
    n_estimators=2000,
    max_depth=70,
    max_features=2,
    min_samples_leaf=4,
    min_samples_split=3,
    bootstrap=True
)

model.fit(X_train,y_train)

#shap
srf = shap.TreeExplainer(model)
sv = srf.shap_values(X_test)

shap.summary_plot(sv[1],X_test,plot_type='bar')
shap.summary_plot(sv[1],X_test)

In [ ]:
def predict(model,patient):
    srf = shap.TreeExplainer(model)
    sv = srf.shap_values(patient)
    shap.initjs
    return shap.force_plot(srf.expected_value[1], sv[1], patient)

#%% 
#predicting patients
patient = X_test.iloc[42,:].astype(float)
predict(model, patient)

patient = X_test.iloc[15,:].astype(float)
predict(model, patient)

patient = X_test.iloc[123,:].astype(float)
predict(model, patient)

sv = srf.shap_values(X_train.iloc[:50])
shap.force_plot(srf.expected_value[1],sv[1],X_test.iloc[:50])